In [1]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader

# Data

In [ ]:
path = r"D:\ResearchInternship_Data\MNIST"
transform = T.Cartesian(cat=False)
train_dataset = MNISTSuperpixels(path, True, transform=transform)
test_dataset = MNISTSuperpixels(path, False, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
d = train_dataset


def normalized_cut_2d(edge_index, pos):
    row, col = edge_index
    edge_attr = torch.norm(pos[row] - pos[col], p=2, dim=1)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0))



# Model

In [ ]:
#https://discuss.pytorch.org/t/any-pytorch-function-can-work-as-keras-timedistributed/1346/4
class TimeDistributed(nn.Module):
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):

        if len(x.size()) <= 2:
            return self.module(x)

        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, 1, x.size(-1))  # (samples * timesteps, channels, input_size)

        y = self.module(x_reshape)

        # We have to reshape Y
        y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)

        return y

In [5]:
class Net(nn.Module):
    def __init__(self, filters):
        super().__init__()
        layers = []
        in_filters = 1
        for filt in filters:
            layers.append(torch.nn.Conv1d(in_filters, filt, 3))
            layers.append(torch.nn.ReLU())
            in_filters = filt
        
        self.conv = nn.Sequential(*layers)
        
        self.gconv = GCNConv(64, 64)

        self.fc1 = torch.nn.Linear(64, 128)
        self.fc2 = torch.nn.Linear(128, d.num_classes)

    def forward(self, x, edge_index, batch):
        # input x shape [batch, time_seg_len, num_nodes, ndims]
        x = self.conv(x)
        # x shape [batch, 1, num_nodes, filters]
        # x shape [batch, num_nodes, filters]
        x = self.gconv1(x, edge_index)
        x = x.relu()
        # x shape [batch, num_nodes, 1, filters]
        x = global_mean_pool(x, batch)
        
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        return F.log_softmax(self.fc2(x), dim=1)

Extracting D:\ResearchInternship_Data\MNIST\raw\MNISTSuperpixels.zip
Processing...
Done!


# Training

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train(epoch):
    model.train()

    if epoch == 16:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001

    if epoch == 26:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        F.nll_loss(model(data), data.y).backward()
        optimizer.step()


def test():
    model.eval()
    correct = 0

    for data in test_loader:
        data = data.to(device)
        pred = model(data).max(1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(test_dataset)


for epoch in range(1, 31):
    train(epoch)
    test_acc = test()
    print(f'Epoch: {epoch:02d}, Test: {test_acc:.4f}')

ImportError: `graclus` requires `torch-cluster`.